In [ ]:
#!pip install -U -q datasets transformers[torch] timm wandb torchmetrics matplotlib albumentations numpy scikit-learn

In [ ]:
import json
import wandb
from transformers import DetrForObjectDetection, DetrImageProcessor, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from data.dataset import AUAIR, collate_fn
from model.evaluate import evaluate_model
from utils.visualize import visualize_labels
from utils.preprocess import filter_invalid_bboxes

In [ ]:
"""
wandb.init(
    project="DI725_assignment_2_2389088_detr-auair"
)
config = wandb.config
"""

In [ ]:
annotations_path = "data/auair2019/annotations.json"
images_path = "data/auair2019/images"

with open(annotations_path, "r") as f:
    annotations = json.load(f)

train_val, test = train_test_split(annotations['annotations'], test_size=0.2, random_state=42)
train, val = train_test_split(train_val, test_size=0.2, random_state=42)

In [ ]:
train_dataset = AUAIR(annotations=train, images_dir=images_path)
val_dataset = AUAIR(annotations=val, images_dir=images_path)
test_dataset = AUAIR(annotations=test, images_dir=images_path)

In [ ]:
id2label = {
    0: "Human",
    1: "Car",
    2: "Truck",
    3: "Van",
    4: "Motorbike",
    5: "Bicycle",
    6: "Bus",
    7: "Trailer"
}
num_classes = 8
label2id = {v: k for k, v in id2label.items()}

In [ ]:
idx = 100  # To visualize different samples, change idx
visualize_labels(train_dataset[idx][0], train_dataset[idx][1], id2label)

In [ ]:
model = DetrForObjectDetection.from_pretrained(
    "facebook/detr-resnet-50",
    num_labels=num_classes,
    ignore_mismatched_sizes=True  # optional if you're changing num_labels
)

processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

In [ ]:
training_args = TrainingArguments(
    output_dir="model/training/detr-finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=10,
    learning_rate=5e-5,
    weight_decay=1e-4,
    logging_dir="./logs",
    logging_steps=1000,
    save_steps=1000,
    save_total_limit=2,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor,
    data_collator=collate_fn,
)

In [ ]:
trainer.train()

In [ ]:
model_path = "model/best_model/detr-finetuned"
processor_path = "model/processor/detr-finetuned"

trainer.save_model(model_path)
processor.save_pretrained(processor_path)

In [ ]:
model_loaded = DetrForObjectDetection.from_pretrained(model_path)
processor_loaded = DetrImageProcessor.from_pretrained(processor_path)

In [ ]:
results = evaluate_model(model_loaded, test_dataset, processor_loaded, id2label, batch_size=1, max_samples=50, show_samples=5)
print("Evaluation Results:", results)